In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"

!pip install forex-python

<img src='../../../common/logo_DH.png' align='left' width=35%/>

# APIs - Práctica Guiada

<a id="section_toc"></a> 
## Tabla de Contenidos

[1- Intro](#section_intro)

[2- `forex_python`](#section_Forex)

$\hspace{.5cm}$[2.1- Currency Rates](#section_currency)

[3- Bitcoin Tracking](#section_bitcoin)

[4- Bitcoin Tracker Continuo](#section_bitcoinscript)

[5- BONUS: Bitcoin Tracker Continuo a CSV](#section_bitcointocsv)


---

<a id="section_intro"></a> 
###  1- Intro
[volver a TOC](#section_toc)

#### ¿Qué es una API?

Como vimos en la introducción teórica, una API es una interfaz que se expone por parte de el equipo propietario de determinada información, con el fin de disponibilizarla para ser consumida programáticamente.

Existen APIs de muchos tipos y para muchas aplicaciones comunes (o no tanto). Este ejemplo ilustra muy sencillamente cómo podemos usar la API de Forex para armar un tracker de Bitcoin, en la moneda que elijamos.

Es importante destacar que dentro de lo que son REST APIs, podemos encontrar aquellas que disponibilizan endpoints para accederse mandando requests genéricas (usando la librería requests, o incluso nuestro navegador para obtener jsons de datos), como también aquellas que tienen su propia librería para consumirse. Este segundo tipo tiene la desventaja que nos limita a lo que esté implementado en la librería, que puede no ser todo lo que la API ofrece, pero tiene la gran ventaja de que nos habilita métodos sencillos y directos para obtener la información.

<a id="section_Forex"></a> 
###  2- `forex_python`
[volver a TOC](#section_toc)

La práctica de hoy usará la librería [forex_python](https://github.com/MicroPyramid/forex-python), que nos permite utilizar la API del European Central Bank con el fin de obtener distintas conversiones entre divisas o incluso criptomonedas. Independientemente del nombre de la librería, no usaremos la plataforma Forex, que requiere una cuenta, ya que nuestro objetivo es un tracker sencillo y no un algoritmo de trading en toda regla.

<a id="section_currency"></a> 
#### 2.1- Currency Rates
[volver a TOC](#section_toc)

Una de las comodidades que ofrece el European Central Bank en su API, es un convertidor de divisas. Vamos a utilizarlo para ver cómo se encuentra el dólar respecto de las principales monedas que intercambia este banco.

Es muy importante notar que esta librería no es oficial, y que por lo tanto no está demasiado bien documentada fuera de los ejemplos que pueden ver en el README en Github.

In [ ]:
from forex_python.converter import CurrencyRates

In [ ]:
# Instanciamos la clase que usa la librería para valores de moneda.
c = CurrencyRates()

Veamos cómo se ubica el Euro respecto de las principales divisas que mueve el European Central Bank. No parece sorprendente no ver el ARS entre ellas:

In [ ]:
# Veamos cómo se ubica el Euro respecto de las principales divisas que mueve
# el European Central Bank. No parece sorprendente no ver el ARS entre ellas.
c.get_rates("")

<a id="section_bitcoin"></a> 
###  3- Bitcoin Tracking
[volver a TOC](#section_toc)

Durante el último par de años, se volvieron extremadamente preponderantes las criptomonedas en los mercados financieros, debido a la facilidad relativa para intercambiar estos activos, y a una enorme volatilidad que permitió algunos arbitrajes realmente fascinantes.

La práctica de hoy se centra específicamente en el Bitcoin, y su precio en distintas monedas, así como la relación entre las mismas.

Usaremos el método `get_latest_price` del tracker de bitcoin, con el fin de ver en "tiempo real" la cotización.

In [ ]:
from forex_python.bitcoin import BtcConverter
b = BtcConverter() # force_decimal=True to get Decimal rates
round(b.get_latest_price('USD'), 2)

Veamos ahora el mismo método, pero usando como moneda el peso argentino:

In [ ]:
round(b.get_latest_price('ARS'), 2)

Teniendo ambos precios, veamos si podemos obtener el valor del "dólar oficial":

In [ ]:
round((b.get_latest_price('ARS'))/b.get_latest_price('USD'), 2)

La API también nos permite efectuar una conversión, al precio actual, de una cantidad de divisa determinada:

In [ ]:
b.convert_to_btc(400, 'USD')

<a id="section_bitcoinscript"></a> 
###  4- Bitcoin Tracking Continuo
[volver a TOC](#section_toc)

Hemos explorado hasta aquí distintas posibilidades que nos permite esta API. Vamos a intentar generar un tracker continuo de bitcoin, que nos devuelva para un dado tiempo la cotización, que efectúe chequeos por minuto, y que tenga una duración máxima opcional.

Noten que si no se le especifica duración máxima, el loop será infinito y habrá que frenarlo manualmente (con el botón de `stop` de la interfaz de Jupyter, o mandando una señal de `KeyboardInterrupt` de algún otro modo).

In [ ]:
import time

def bitcoin_tracking(currency="USD", time_between_values=1, total_times=None):
    """
    Función para rastrear el tiempo en bitcoins.
    
    Parameters
    ----------
    currency : str
        Código de la moneda a usar. Por default 'USD'.
    time_between_values: int
        Cantidad de minutos entre lecturas. Default cada un minuto.
    total_times: int or None
        Cantidad máxima de lecturas que se harán si no ocurre un error. Por default
        loop infinito.
    """
    step = 1
    b = BtcConverter()
    while True:
        price = round(b.get_latest_price(currency), 2)
        print(price, currency, "for 1 bitcoin")
        if total_times is not None:
            if step == total_times:
                break
            else:
                step += 1
        time.sleep(60 * time_between_values)
    print(f"Ejecutada la lectura {total_times} veces.")

In [ ]:
# WARNING: EJECUTAR ESTA CELDA IMPIDE CONTINUAR EJECUTANDO POR 2 MINUTOS!
bitcoin_tracking(total_times=2)

<a id="section_bitcointocsv"></a> 
###  5- BONUS: Bitcoin Tracking Continuo a CSV
[volver a TOC](#section_toc)

Vamos a mejorar nuestro tracker para poder almacenar la data en formato csv, sin depender de pandas (para no colmar la memoria de nuestra computadora, aunque para eso tardaríamos mucho tiempo).

Notar que lo ideal para este tipo de funciones sería correrse con un script desde terminal, para evitar tener activo un kernel de Jupyter (pueden guardar esta función con los imports pertinentes en un archivo `.py` y probarlo, pero para la clase lo vamos a correr desde la notebook).

In [ ]:
import csv
import datetime

def bitcoin_tracking_csv(currency="USD", time_between_values=1, total_times=None):
    """
    Función para rastrear el tiempo en bitcoins.
    
    Parameters
    ----------
    currency : str
        Código de la moneda a usar. Por default 'USD'.
    time_between_values: int
        Cantidad de minutos entre lecturas. Default cada un minuto.
    total_times:
        Cantidad máxima de lecturas que se harán si no ocurre un error. Por default
        loop infinito.
    """
    lines = 1
    b = BtcConverter()
    date = datetime.datetime.now()
    fname = f"../Data/Bitcoin-{date.strftime('%Y%m%d')}.csv"
    with open(fname, "w", newline="") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=["date", f"bitcoin_price_{currency}"])
        writer.writeheader()
        while True:
            try:
                price = b.get_latest_price(currency)
                date = datetime.datetime.now()
                dict_price = {
                    "date": date.strftime("%Y-%m-%d %H:%M:%S"),
                    f"bitcoin_price_{currency}": price
                }
                writer.writerow(dict_price)
                if total_times is not None:
                    if lines == total_times:
                        print(f"Ejecutada la lectura {total_times} veces.")                        
                        break
                lines += 1
                time.sleep(60 * time_between_values)
            except KeyboardInterrupt:
                print(f"Ejecución interrumpida. {lines} líneas registradas.")
                break

In [ ]:
bitcoin_tracking_csv()